In [26]:
import re
from collections import defaultdict
from typing import Dict, List, Tuple
from tqdm import tqdm

In [27]:
# 불용어 제거
SPECIALS = "".join([".", ",", ";" ,":", "!","?", "'", '"', " " ])

In [28]:
def preprocess(text: str, only_kor: bool=True):
  """ 한국어 문장을 옵션에 맞게 전처리 """
  # 한국어 모음과 특수 문자, 숫자 및 영어 제거
  if only_kor:
    text = re.sub(f"[^가-힣| |]+", "", text)
  else:
    text = re.sub(f"[^가-힣|ㄱ-ㅎ|0-9|{SPECIALS}|a-zA-Z]+", "", text)

  # 연속 공백 제거
  text = re.sub(" +", "", text)

  # 좌우 불필요한 공백 제거
  return text.strip()

sent = "ㅋㅋㅋㅋㅋㅋ 안녕하세요 ! \"저는\" cola를 좋아합니다."

preprocess(sent)
preprocess(sent, only_kor=False)

'ㅋㅋㅋㅋㅋㅋ안녕하세요!"저는"cola를좋아합니다.'

# 1. get_vocab

In [29]:
def get_vocab(f_name: str) -> Dict[str, int]:
  """ 코퍼스 파일을 읽어와 단어 사전 구축 """
  vocab = defaultdict(int)
  with open(f_name, "r", encoding="utf-8") as corpus:
    for line in corpus:
      tokens = preprocess(line).strip().split()
      for token in tokens:
        vocab[" ".join(list(token)) + "</w>"] += 1
  return dict(vocab)

In [30]:
# 더미 데이터로 실험 하기 위한 함수 구현
def pseudo_get_vocab(corpus: List[str]) -> Dict[str, int]:
  """더미 데이터를 읽어와 단어 사전 구축"""
  vocab = defaultdict(int)
  for line in corpus:
    tokens = preprocess(line).strip().split(" ")
    for token in tokens:
      vocab[" ".join(list(token)) + " </w>"] += 1
  return dict(vocab)

In [40]:
# 원하는 문장 넣기
corpus = [
    "오늘 점심에 배가 너무 고파서 밥을 너무 많이 먹었다.",
    "오늘 점심에 배가 고파서 밥을 많이 먹었다.",
    "오늘 배가 너무 고파서 점심에 밥을 너무 많이 먹었다.",
    "오늘 점심에 배가 고파서 버스를 많이 먹었다.",
    "펩시는 사랑하지 않아",
    "어제 저녁에 밥을 너무 많이 먹었더니 배가 부르다.",
    "이따가 오후 7시에 출발하는 비행기가 3시간 연착 되었다고 하네요."
]


In [32]:
vocab = pseudo_get_vocab(corpus)
vocab

{'나 는 나 를 사 랑 해 </w>': 1,
 '나 는 너 를 사 랑 해 </w>': 1,
 '콜 라 가 좋 아 </w>': 1,
 '딸 기 는 사 랑 하 지 않 아 </w>': 1,
 '펩 시 는 사 랑 하 지 않 아 </w>': 1,
 '네 가 사 랑 하 는 나 </w>': 1,
 '내 가 사 랑 하 는 너 </w>': 1}

In [33]:
# 구축된 사전을 순회하며 사전 내 등록된 캐릭터 토큰과 등장 횟수를 반환
def get_tokens(vocab: Dict[str, int]):
  """사전 내 등록된 토큰을 확인"""
  result = defaultdict(int)
  for word, freq in vocab.items():
    tokens = word.split()
    for token in tokens:
      result[token] += freq

  return dict(result)

In [34]:
tokens = get_tokens(vocab)
tokens

{'나': 4,
 '는': 6,
 '를': 2,
 '사': 6,
 '랑': 6,
 '해': 2,
 '</w>': 7,
 '너': 2,
 '콜': 1,
 '라': 1,
 '가': 3,
 '좋': 1,
 '아': 3,
 '딸': 1,
 '기': 1,
 '하': 4,
 '지': 2,
 '않': 2,
 '펩': 1,
 '시': 1,
 '네': 1,
 '내': 1}

In [35]:
# 자주 등장한 페어를 구하기
def get_stats(vocab: Dict[str, int]):
  """사전을 활용한 바이그램 페어 구축"""
  pairs = defaultdict(int)
  for word, freq, in vocab.items():
    symbols = word.split()
    for i in range(len(symbols)-1):
      pairs[symbols[i], symbols[i+1]] += freq
  return dict(pairs)

In [37]:
pairs = get_stats(vocab)
pairs

{('나', '는'): 2,
 ('는', '나'): 2,
 ('나', '를'): 1,
 ('를', '사'): 2,
 ('사', '랑'): 6,
 ('랑', '해'): 2,
 ('해', '</w>'): 2,
 ('는', '너'): 2,
 ('너', '를'): 1,
 ('콜', '라'): 1,
 ('라', '가'): 1,
 ('가', '좋'): 1,
 ('좋', '아'): 1,
 ('아', '</w>'): 3,
 ('딸', '기'): 1,
 ('기', '는'): 1,
 ('는', '사'): 2,
 ('랑', '하'): 4,
 ('하', '지'): 2,
 ('지', '않'): 2,
 ('않', '아'): 2,
 ('펩', '시'): 1,
 ('시', '는'): 1,
 ('네', '가'): 1,
 ('가', '사'): 2,
 ('하', '는'): 2,
 ('나', '</w>'): 1,
 ('내', '가'): 1,
 ('너', '</w>'): 1}

In [38]:
def merge_vocab(pair: Tuple[str, str],vocab: Dict[str, int]):
  """가장 자주 등장한 바이그램 페어를 엮어줌"""
  result = defaultdict(dict)
  for word in vocab:
    paired = word.replace(" ".join(pair), "".join(pair))
    result[paired] = vocab[word]
  return dict(result)

In [39]:
num_merges = 5

for i in range(num_merges):
  pairs = get_stats(vocab)
  if not pairs:
    break
  best = max(pairs, key=pairs.get)
  vocab = merge_vocab(best, vocab)
  tokens = get_tokens(vocab)
  print(f"lter: {i+1}\n'"
        f"Best pair: {best}\n"
        f"Tokens: {tokens}\n"
        f"Number of tokens: {len(tokens)}\n")


lter: 1
'Best pair: ('사', '랑')
Tokens: {'나': 4, '는': 6, '를': 2, '사랑': 6, '해': 2, '</w>': 7, '너': 2, '콜': 1, '라': 1, '가': 3, '좋': 1, '아': 3, '딸': 1, '기': 1, '하': 4, '지': 2, '않': 2, '펩': 1, '시': 1, '네': 1, '내': 1}
Number of tokens: 21

lter: 2
'Best pair: ('사랑', '하')
Tokens: {'나': 4, '는': 6, '를': 2, '사랑': 2, '해': 2, '</w>': 7, '너': 2, '콜': 1, '라': 1, '가': 3, '좋': 1, '아': 3, '딸': 1, '기': 1, '사랑하': 4, '지': 2, '않': 2, '펩': 1, '시': 1, '네': 1, '내': 1}
Number of tokens: 21

lter: 3
'Best pair: ('아', '</w>')
Tokens: {'나': 4, '는': 6, '를': 2, '사랑': 2, '해': 2, '</w>': 4, '너': 2, '콜': 1, '라': 1, '가': 3, '좋': 1, '아</w>': 3, '딸': 1, '기': 1, '사랑하': 4, '지': 2, '않': 2, '펩': 1, '시': 1, '네': 1, '내': 1}
Number of tokens: 21

lter: 4
'Best pair: ('나', '는')
Tokens: {'나는': 2, '나': 2, '를': 2, '사랑': 2, '해': 2, '</w>': 4, '너': 2, '콜': 1, '라': 1, '가': 3, '좋': 1, '아</w>': 3, '딸': 1, '기': 1, '는': 4, '사랑하': 4, '지': 2, '않': 2, '펩': 1, '시': 1, '네': 1, '내': 1}
Number of tokens: 22

lter: 5
'Best pair: ('를', '사랑')
Token

### 결론
BPE 를 활용한 tokenizer는 최장 길이 토큰의 매칭을 우선적으로 적용 -> 사전을 단어 길이 기준의 내림차순으로 정렬